In [ ]:
#123
import pandas as pd
#aabbccdee
df = pd.read_csv("water_potability.csv")

In [ ]:
from matplotlib import pyplot as plt

df.hist(figsize=(15, 15))
plt.show()

In [ ]:
from numpy import true_divide
import seaborn as sns

plt.figure(figsize=(8, 8))
sns.heatmap(df.corr(), annot=True, linewidths=1, square=True)

In [ ]:
df.isnull().sum()

In [ ]:
df["ph"].fillna(value=df["ph"].median(), inplace=True)
df["Sulfate"].fillna(value=df["Sulfate"].median(), inplace=True)
df["Trihalomethanes"].fillna(value=df["Trihalomethanes"].median(), inplace=True)
df.isnull().sum()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

lr=LogisticRegression()
tree=DecisionTreeClassifier()
forest=RandomForestClassifier()
svm=SVC()
knn=KNeighborsClassifier()

In [ ]:
#決策樹
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import plot_tree
import seaborn as sns
import matplotlib.pyplot as plt

# 切分特徵和目標變量
X = df.drop("Potability", axis=1)
y = df["Potability"]

# 切分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

DecisionTree_model_param={'max_depth':[i for i in range(1,101)],'max_features':[i for i in range(1, len(X.columns) + 1)]}
DecisionTree_grid_search=GridSearchCV(DecisionTreeClassifier(),DecisionTree_model_param,cv=5)
DecisionTree_grid_search.fit(X_train,y_train)
print(DecisionTree_grid_search.best_params_)
print(DecisionTree_grid_search.best_score_)
best_param=DecisionTree_grid_search.best_params_

#建立決策樹模型
DecisionTree_model = DecisionTreeClassifier(max_features=best_param['max_features'],max_depth=best_param['max_depth'])
DecisionTree_model.fit(X_train, y_train)

#進行預測
y_pred = DecisionTree_model.predict(X_test)

# 評估模型性能
DecisionTree_accuracy = accuracy_score(y_test, y_pred)
print(f"\nDecisionTree Accuracy: {DecisionTree_accuracy}\n")

# 分類報告
DecisionTree_class_report = classification_report(y_test, y_pred)
print("DecisionTree Classification Report:")
print(DecisionTree_class_report)

# 混淆矩陣
DecisionTree_conf_matrix = confusion_matrix(y_test, y_pred)

# 使用Seaborn來繪製熱力圖
plt.figure(figsize=(5, 3))
sns.heatmap(DecisionTree_conf_matrix, annot=True, fmt='d', cmap='YlOrBr', cbar=False,
            xticklabels=["Non-Potable", "Potable"],
            yticklabels=["Non-Potable", "Potable"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('DecisionTree Confusion Matrix')
plt.show()

# 特徵名稱
feature_names = X.columns.tolist()

# 可視化決策樹
plt.figure(figsize=(15, 10))
plot_tree(DecisionTree_model, feature_names=feature_names, class_names=["Non-Potable", "Potable"], filled=True, rounded=True)
plt.show()

# 交叉驗證
cross_val_scores = cross_val_score(DecisionTree_model, X, y, cv=5)
print(f"Cross-Validation Scores: {cross_val_scores}")
print(f"Mean Accuracy: {cross_val_scores.mean()}")

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 使用網格搜索尋找最佳K值
model_param = {'n_neighbors': range(1, 21)}
grid_search = GridSearchCV(KNeighborsClassifier(), model_param, cv=5)
grid_search.fit(X_train_scaled, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
best_k = grid_search.best_params_['n_neighbors']

# 使用最佳K值擬合新的KNN模型
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train_scaled, y_train)

# 預測
y_pred_best = knn.predict(X_test_scaled)

# 評估模型性能
accuracy = accuracy_score(y_test, y_pred_best)
classification_rep = classification_report(y_test, y_pred_best)

# 分類報告
print(f'\nKNN Accuracy: {accuracy}\n')
print(f'KNN Classification Report:\n{classification_rep}')

# 使用Seaborn來繪製混淆矩陣
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(5, 3))
sns.heatmap(conf_matrix_best, annot=True, fmt='d', cmap='YlOrBr', cbar=False,
            xticklabels=["Non-Potable", "Potable"],
            yticklabels=["Non-Potable", "Potable"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('KNN Confusion Matrix')
plt.show()

# 交叉驗證
cv_val_scores = cross_val_score(knn, X, y, cv=5)
print(f"\nCross-Validation Scores: {cv_val_scores}")
print(f"Mean Accuracy: {cv_val_scores.mean()}")



# 取Hardness和Organic_carbon特徵用於可視化
'''如果水中鈣和鎂離子的濃度較高，則水的硬度也相對較高
水體来源：
如果水體源自相似的地質區域，地下水中的礦物質可能同時影響硬度和有機碳的濃度。例如，地質中豐富的碳酸鈣可能提高水的硬度，同時地下水流經有機質豐富的土壤可能導致水中有機碳的增加。

人為活動：
農業和工業活動可能同時導致水中有機碳和硬度的變化。例如，農業過程中使用的肥料可能引入有機物和養分，同時礦物的流入也可能提高水的硬度。

自然處理過程：
自然的地下水循環和生態系統的相互作用可能導致硬度和有機碳的變化。地下水流通過地層時可能溶解礦物質，影響硬度，同時地表水中的生態作用和分解可能影響有機碳的濃度。

水體中的微生物活動：
水中的微生物可能同時受到硬度和有機碳的影響。有機碳提供了微生物生長的營養源，而硬度可能影響某些微生物的生態。
'''
import numpy as np

X_2d = X[['Hardness', 'Organic_carbon']]

# 設定模型和參數
knn_vis = KNeighborsClassifier(n_neighbors=best_k)

# 訓練模型
knn_vis.fit(X_2d, y)

# 繪製決策邊界
h = .02
x_min, x_max = X_2d.iloc[:, 0].min() - 1, X_2d.iloc[:, 0].max() + 1
y_min, y_max = X_2d.iloc[:, 1].min() - 1, X_2d.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = knn_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# 繪製數據點和決策邊界
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
plt.scatter(X_2d.iloc[:, 0], X_2d.iloc[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k', s=50)
plt.xlabel('Hardness')
plt.ylabel('Organic_carbon')
plt.title(f'KNN Decision Boundaries (k={best_k})')
plt.show()